<a href="https://colab.research.google.com/github/champsleague/DeepLearning/blob/main/DL_AL02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
import os
import shutil

# Set the directory where your images are located
image_dir = '/content/gdrive/MyDrive/Chinese food'

# Set the directory to store the labeled data
labeled_data_dir = '/content/gdrive/MyDrive/labeled_data'
os.makedirs(labeled_data_dir, exist_ok=True)

# List all files in the image directory
image_files = os.listdir(image_dir)

# Iterate through the image files
for file in image_files:
    # Extract the class label from the filename
    class_label = file.split('_')[0]  # Assuming the format is "class_label_filename.png"
    
    # Create the subdirectory for the class label if it doesn't exist
    class_dir = os.path.join(labeled_data_dir, class_label)
    os.makedirs(class_dir, exist_ok=True)
    
    # Move the image file to the corresponding class subdirectory
    file_path = os.path.join(image_dir, file)
    new_file_path = os.path.join(class_dir, file)
    shutil.copyfile(file_path, new_file_path)
    
    # Create a text file with the same name as the image file and write the label
    text_file_path = os.path.join(class_dir, os.path.splitext(file)[0] + '.txt')
    with open(text_file_path, 'w') as f:
        f.write(class_label)


In [11]:
import os
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from PIL import Image

# Set the directory where your labeled data is located
labeled_data_dir = '/content/gdrive/MyDrive/labeled_data'

# Set the directory to save the augmented and preprocessed images
augmented_images_dir = '/content/gdrive/MyDrive/augmented_images'
os.makedirs(augmented_images_dir, exist_ok=True)

# Set the maximum number of labeled images to use
max_labeled_images = 40

# Create a list of labeled image filenames
labeled_image_files = []
for class_label in os.listdir(labeled_data_dir):
    class_dir = os.path.join(labeled_data_dir, class_label)
    if os.path.isdir(class_dir):
        filenames = os.listdir(class_dir)
        for filename in filenames:
            # Check if the file is an image (supports common image file extensions)
            if filename.endswith('.png') or filename.endswith('.jpg') or filename.endswith('.jpeg'):
                labeled_image_files.append(os.path.join(class_dir, filename))

# Randomly select a subset of labeled images
random.seed(42)
random.shuffle(labeled_image_files)
labeled_image_files = labeled_image_files[:max_labeled_images]

# Create an instance of the ImageDataGenerator with desired augmentation options
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input
)

# Perform data augmentation and preprocessing
num_augmented_images = 200

for i in range(num_augmented_images):
    # Randomly select a labeled image from the subset
    image_path = random.choice(labeled_image_files)
    image = Image.open(image_path)
    
    # Preprocess and augment the image
    image = image.resize((224, 224))
    image = preprocess_input(np.array(image))
    image = np.expand_dims(image, axis=0)

    augmented_images = datagen.flow(image, batch_size=1, save_to_dir=augmented_images_dir, save_prefix='augmented_')
    next(augmented_images)
